#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [3]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [32]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')
#es el mismo data set

#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [5]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [6]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


In [9]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


###Separamos los datos por 4 categorias {‘dog’,’cat’,’automobile’,bird’}

In [14]:
graphlab.canvas.set_target('ipynb')

In [53]:
image_train_dog=image_train[image_train['label']=='dog']
image_train_cat=image_train[image_train['label']=='cat']
image_train_auto=image_train[image_train['label']=='automobile']
image_train_bird=image_train[image_train['label']=='bird']

In [24]:
print(image_train_dog['image'].show(),image_train_cat['image'].show(),
     image_train_auto['image'].show(),image_train_bird['image'].show())

(None, None, None, None)


###Hacemos lo mismo pero con los datos de prueba

In [54]:
image_test_dog=image_test[image_test['label']=='dog']
image_test_cat=image_test[image_test['label']=='cat']
image_test_auto=image_test[image_test['label']=='automobile']
image_test_bird=image_test[image_test['label']=='bird']

In [55]:
print(image_test_dog['image'].show(),image_test_cat['image'].show(),
     image_test_auto['image'].show(),image_test_bird['image'].show())

(None, None, None, None)


#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [25]:
#es la misma idea de lo que se hizo con la similitud de documentos 'document retrieval'
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [26]:
dog_model = graphlab.nearest_neighbors.create(image_train_dog,features=['deep_features'],label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [27]:
cat_model = graphlab.nearest_neighbors.create(image_train_cat,features=['deep_features'],label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [28]:
auto_model = graphlab.nearest_neighbors.create(image_train_auto,features=['deep_features'],label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [29]:
bird_model = graphlab.nearest_neighbors.create(image_train_bird,features=['deep_features'],label='id')

PROGRESS: Starting brute force nearest neighbors model training.


#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [5]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()
#definimos una imagen en particular asi como lo hicimos con obama

###funcion para busqueda de referencias

In [62]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')
#regresame todas la imagenes que matcheen con las referencs_labels

In [7]:
knn_model.query(cat)
#vemos que el reference_label es'la imagen' por lo que se va a querer transformar el feature

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.97ms       |
PROGRESS: | Done         |         | 100         | 62.996ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


###Ahora vemos para los casos particulares de las categorias

In [35]:
cat1=image_test[0:1]
cat1['image'].show()

In [46]:
query_cat1=cat_model.query(cat1)
query_cat1
#los k cercanos de cat1

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 21.913ms     |
PROGRESS: | Done         |         | 100         | 148.727ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [50]:
#distancia promedio de los 5 mas cercanos
query_cat1['distance'].mean()

36.15573070978294

In [67]:
cat1_cat1=image_train_cat[image_train_cat['id']==16289]
cat1_cat1['image'].show()

###algo mejor, cual es el perro mas cercano al gatito

In [51]:
query2_cat1=dog_model.query(cat1)
query2_cat1

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 45.669ms     |
PROGRESS: | Done         |         | 100         | 179.529ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [52]:
#distancia promedio de los 5 mas cercanos
query2_cat1['distance'].mean()

37.77071136184157

In [45]:
dog1=image_train_dog[image_train_dog['id']==16976]
dog1['image'].show()
#el perrito mas parecido a un gato :3

In [58]:
dog_model.query(image_test,k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 2
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 4000         | 1016000 | 49.9018     | 3.03s        |
PROGRESS: | Done         | 2036000 | 100         | 3.25s        |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16976,37.4642628784,1
1,3828,42.3025826356,1
2,40938,29.3472319585,1
3,1401,43.1386196575,1
4,49803,33.4773590373,1
5,5755,32.8458495684,1
6,20715,35.0397073189,1
7,47496,50.5112353783,1
8,13387,33.9010327697,1
9,12089,37.4849250909,1


###distancia de perros con las demas categorias

In [65]:
dog_auto_neighbors = auto_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 2
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 255000  | 50.0982     | 1.12s        |
PROGRESS: | Done         | 509000  | 100         | 1.21s        |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 2
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 239000  | 50          | 971.423ms    |
PROGRESS: 

In [66]:
dog_auto_neighbors

query_label,reference_label,distance,rank
0,33859,41.9579761457,1
1,2046,46.0021331807,1
2,19594,42.9462290692,1
3,11000,41.6866060048,1
4,19594,39.2269664935,1
5,49314,40.5845117698,1
6,40822,45.1067352961,1
7,44997,41.3221140974,1
8,33859,41.8244654995,1
9,33859,45.4976929401,1


In [64]:
#hace la busqueda del query por id
get_images_from_ids(dog_cat_neighbors)

id,image,label,deep_features,image_array
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
331,Height: 32 Width: 32,cat,"[0.0, 0.0,0.510963916779, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
367,Height: 32 Width: 32,cat,"[1.38658058643, 0.0, 0.0,0.0, 0.0, 0.182891070 ...","[168.0, 151.0, 143.0,145.0, 130.0, 124.0, ..."
494,Height: 32 Width: 32,cat,"[0.0, 0.0539512038231,1.95745122433, 0.0, 0.0, ...","[26.0, 34.0, 29.0, 24.0,29.0, 25.0, 33.0, 43.0, ..."
597,Height: 32 Width: 32,cat,"[0.0, 0.0,0.0470637083054, 0.0, ...","[133.0, 153.0, 138.0,126.0, 146.0, 136.0, ..."
882,Height: 32 Width: 32,cat,"[0.0, 0.0,0.156200289726, 0.0, ...","[141.0, 133.0, 112.0,143.0, 133.0, 113.0, ..."
1252,Height: 32 Width: 32,cat,"[1.57850754261, 0.0, 0.0,0.0, 0.0, 0.0, ...","[217.0, 209.0, 207.0,216.0, 208.0, 205.0, ..."
1696,Height: 32 Width: 32,cat,"[0.0, 0.0,0.109984338284, 0.0, ...","[12.0, 12.0, 12.0, 18.0,18.0, 18.0, 58.0, 58.0, ..."
1846,Height: 32 Width: 32,cat,"[0.0, 0.00756752490997,0.0, 0.523695707321, ...","[60.0, 69.0, 42.0, 54.0,66.0, 38.0, 71.0, 99.0, ..."


We are going to create a simple function to view the nearest neighbors to save typing:

In [61]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')
#regresame todas la imagenes que matcheen con las referencs_labels

In [10]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))
#encontrar los k-vecinos
#ver el query es el mismo que arriba

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.302ms      |
PROGRESS: | Done         |         | 100         | 69.934ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [11]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [12]:
car = image_train[8:9]
car['image'].show()
#hacemos lo mismo para los carros

In [13]:
get_images_from_ids(knn_model.query(car))['image'].show()
#aqui aparecen los similares

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 26.12ms      |
PROGRESS: | Done         |         | 100         | 70.419ms     |
PROGRESS: +--------------+---------+-------------+--------------+


#Just for fun, let's create a lambda to find and show nearest neighbor images

In [14]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()
#lo quiere hacer para todas las entradas de la tabla de imagenes i(indice de la tabla)
es la funcion que generaliza los ejercicios

In [15]:
show_neighbors(8)
#por eso, el numero que le des es como si hiciera la consulta siempre

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 16.302ms     |
PROGRESS: | Done         |         | 100         | 72.25ms      |
PROGRESS: +--------------+---------+-------------+--------------+


In [16]:
show_neighbors(26)
#el numero es el numero de la imagen

PROGRESS: Starting pairwise querying...
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 14.465ms     |
PROGRESS: | Done         |         | 100         | 69.576ms     |
PROGRESS: +--------------+---------+-------------+--------------+
